# Using `numba.jit` to speedup the computation of the Euclidean distance matrix 

Let's implement a function to compute the Euclidean distance matrix using Numba's *just-in-time* compilation decorator and then compare it with the NumPy function we wrote before.

Let's use two Numba functions here: The decorator ` @numba.jit` and `numba.prange`.

In [1]:
import numpy as np
import numba

In [2]:
@numba.jit(nopython=True)
def euclidean_numba1(x, y):
    #Implementation with numba

    num_samples, num_feat = x.shape
    dist_matrix = np.zeros((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(num_samples):
            r = 0.0
            for k in numba.prange(num_feat):
                r += (x[i][k] - y[j][k])**2
            dist_matrix[i][j] = r

    return dist_matrix


@numba.jit(nopython=True)
def euclidean_numba2(x, y):
    #Implementation with numba and numpy

    num_samples, num_feat = x.shape
    dist_matrix = np.zeros((num_samples, num_samples))
    for i in range(num_samples):
        for j in numba.prange(num_samples):
            dist_matrix[i][j] = ((x[i] - y[j])**2).sum()

    return dist_matrix

Let's include here our numpy implementation for comparison.

In [3]:
def euclidean_numpy(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N, m) numpy array
    y: (N, m) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = (x_ij - y_ij)^2
    """

    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[:, np.newaxis].T

    xy = np.dot(x, y.T)

    return np.abs(x2 + y2 - 2. * xy)

The inner loop, which is a reduction, is done with `numba.prange`. `numba.prange` automatically takes care of data privatization and reductions.

In [4]:
# Let's check that they all give the same result
a = 10. * np.random.random([100, 10])

print(np.abs(euclidean_numpy(a, a) - euclidean_numba1(a, a)).max())
print(np.abs(euclidean_numpy(a, a) - euclidean_numba2(a, a)).max())

3.410605131648481e-13
3.410605131648481e-13


Our Numba implementations can be faster than the NumPy one for a list of small vectors. However, with larger vectors, the NumPy implementation is faster:

In [5]:
nsamples = 100
nfeat = 3

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba1(x, x)
%timeit euclidean_numba2(x, x)

117 µs ± 11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
36.7 µs ± 1.88 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2.13 ms ± 49.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
nsamples = 100
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba1(x, x)
%timeit euclidean_numba2(x, x)

128 µs ± 5.01 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
488 µs ± 5.14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.99 ms ± 60 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In a more realistic case, NumPy implementation is much faster:

In [7]:
nsamples = 5000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba1(x, x)

957 ms ± 56.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.37 s ± 20.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
